In [64]:
import sys
import time
import math
import copy
import random
import string
import glob
import numpy as np
import nltk.sentiment.util
from tqdm import tqdm
from os import listdir
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

dataset = "../TermProject/txt_sentoken"
stemming = False
lowerCase = True
stop = True
alpha = True
negation = True
laplace = 1.0
posPrior = 0.8

In [65]:
def loadReviewNames(posDir, negDir):
    positive = glob.glob(posDir)
    negative = glob.glob(negDir)
    print(positive)
    print(negative)
    return positive, negative

In [66]:
#loads dataset in a way that will work with scikit
def getPosNegReviews(directory):
    positive, negative = loadReviewNames(directory + '/pos/*',directory + '/neg/*')
    random.shuffle(positive)
    boundaryTrain = math.floor(0.8 * len(positive))
    trainPos = positive[:boundaryTrain]
    random.shuffle(negative)
    trainNeg = negative[:boundaryTrain]
    
    testPos = positive[boundaryTrain:]
    testNeg = negative[boundaryTrain:]
    return trainPos,trainNeg,testPos,testNeg

In [67]:
def loadDir(name,stemming,lower_case):
    # Loads the files in the folder and returns a list of lists of words from the text in each file
    if stemming:
        porter_stemmer = PorterStemmer()
    tokenizer = RegexpTokenizer(r'\w+')
    data = []
    count = 0
    for f in tqdm(listdir(name)):
        fullname = name+f
        text = []
        with open(fullname, 'rb') as f:
            for line in f:
                if lower_case:
                    line = line.decode(errors='ignore').lower()
                    text += tokenizer.tokenize(line)
                else:
                    text += tokenizer.tokenize(line.decode(errors='ignore'))
        if stemming:
            for i in range(len(text)):
#                 if text[i] in bad_words:
#                     continue
                text[i] = porter_stemmer.stem(text[i])
        data.append(text)
        count = count + 1
    return data

In [68]:
#loads dataset in a way that will work with my unigram Naive bayes implementation
def loadDatasetNB(directory, stemming, lower_case):
    positive = loadDir(directory + '/pos/',stemming, lower_case)
    negative = loadDir(directory + '/neg/',stemming, lower_case)
    random.shuffle(positive)
    boundaryTrain = math.floor(0.8 * len(positive))
    trainPos = positive[:boundaryTrain]
    random.shuffle(negative)
    trainNeg = negative[:boundaryTrain]
    combinedTrain = trainPos + trainNeg
    length = len(trainPos) + len(trainNeg)
    labelsTrain = len(trainNeg) * [1] + len(trainNeg) * [0]
    labelsTrain = np.array(labelsTrain)

    testPos = positive[boundaryTrain:]
    testNeg = negative[boundaryTrain:]
    combinedTest = testPos + testNeg
    labelsTest = len(testPos) * [1] + len(testNeg) * [0]
    labelsTest = np.array(labelsTest)
    return combinedTrain, labelsTrain, combinedTest, labelsTest

In [69]:
    ############  running Naive Bayes #############

In [70]:
#helper function for unigram Naive bayes implementation
def bagOfWordsNB(train_set, train_labels):
    stops = stopwords.words('english') + list(string.punctuation)
    mydict = {}
    smoothing_parameter = 0.034
    posV = 0
    negV = 0
    totalposwords = 0
    totalnegwords = 0

    start = time.process_time()
#     print("creating occurences and wordlist")

    #create bag of words and number of occurences
    count = 0
    for x in train_set:
        rating = train_labels[count]
        count += 1
        if(rating):
            for y in x:
                if y not in mydict and y not in stops:
                    mydict[y] = [1,0] #default [1 pos, 0 neg]
                    posV += 1
                    totalposwords += 1
                elif y not in stops:
                    if mydict[y][0] == 0:
                        posV += 1
                    mydict[y][0] += 1
                    totalposwords += 1
        else:
            for y in x:
                if y not in mydict and y not in stops:
                    mydict[y] = [0,1] #default [0 pos, 1 neg]
                    negV += 1
                    totalnegwords += 1
                elif y not in stops:
                    if mydict[y][1] == 0:
                        negV += 1
                    mydict[y][1] += 1
                    totalnegwords += 1
#     print("review count is: ", count)
#     print("posV", posV)
#     print("negV", negV)
#     print("total word count is:", totalposwords + totalnegwords)
    BOW = mydict, posV, negV, totalposwords, totalnegwords
    return BOW

In [71]:
def naiveBayes(train_set, train_labels, dev_set, smoothing_parameter, pos_prior):
    #Baseline#
    # return predicted labels of development set
    # print("not even started yet")
    smoothing_parameter = 0.034

#     start = time.process_time()

#     print("Going through train set took: ", time.process_time() - start)
    mydict, posV, negV, totalposwords, totalnegwords = bagOfWordsNB(train_set, train_labels)
    
    
    #come up with the bag of words unigram model
    probWordPos = {}
    probWordNeg = {}

    for x in mydict:
        #use laplace smoothing
        # count(W) + a / n + a * (V+1)
        # n = number of words in our training data
        # count(W) = number of times W appeared in training data
        # α is a tuning constant between 0 and 1 (typically small)
        # V = number of word TYPES seen in training data

        probWordPos[x] = math.log((mydict[x][0] + smoothing_parameter) / (totalposwords + smoothing_parameter * (posV + 1)))
        probWordNeg[x] = math.log((mydict[x][1] + smoothing_parameter) / (totalnegwords + smoothing_parameter * (negV + 1)))
        
    start = time.process_time()

    # dev set
    predictions = []
    for x in range(len(dev_set)):
        chancePos = math.log(pos_prior)
        chanceNeg = math.log(1-pos_prior)
        for y in range(len(dev_set[x])):
            if dev_set[x][y] in mydict:
                chancePos += probWordPos[dev_set[x][y]]
                chanceNeg += probWordNeg[dev_set[x][y]]
            # else:
                # chancePos += math.log((smoothing_parameter) / (totalposwords + smoothing_parameter * (posV + 1)))
                # chanceNeg += math.log((smoothing_parameter) / (totalnegwords + smoothing_parameter * (negV + 1)))
        if(chancePos > chanceNeg):
            predictions.append(1)
        else:
            predictions.append(0)
#     print("devset time took:", time.process_time() - start)
    return predictions

In [72]:
def compute_accuracies(predictedLabels, dev_set, dev_labels):
    yhats = predictedLabels
    accuracy = np.mean(yhats == dev_labels)
    tp = np.sum([yhats[i] == dev_labels[i] and yhats[i] == 1 for i in range(len(yhats))])
    precision = tp / np.sum([yhats[i] == 1 for i in range(len(yhats))])
    recall = tp / (np.sum([yhats[i] != dev_labels[i] and yhats[i] == 0 for i in range(len(yhats))]) + tp)
    f1 = 2 * (precision * recall) / (precision + recall)
    return accuracy, f1, precision, recall

In [73]:
numberOfRuntimes = 3
##Naive bayes
accuracyNB = []
f1NB = []
precisionNB = []
recallNB = []
for i in range(numberOfRuntimes):    
    trainSet, trainLabels, revSet, revLabels = loadDatasetNB(dataset, stemming, lowerCase)
    predictedLabels = naiveBayes(trainSet, trainLabels, revSet, laplace, posPrior)
    NBscores = compute_accuracies(predictedLabels, revSet, revLabels)
    accuracyNB.append(NBscores[0])
    f1NB.append(NBscores[1])
    precisionNB.append(NBscores[2])
    recallNB.append(NBscores[3])
#     print("RUN NUMBER " + str(i+1) + " ---------------")
#     print("Accuracy:",NBscores[0])
#     print("F1-Score:",NBscores[1])
#     print("Precision:",NBscores[2])
#     print("Recall:",NBscores[3])

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3255.45it/s]


In [77]:
    aveAccuracy = np.mean(accuracyNB)
    avef1 = np.mean(f1NB)
    avePrecision = np.mean(precisionNB)
    aveRecall = np.mean(recallNB)
    stdAccuracy = np.std(accuracyNB)
    stdf1 = np.std(f1NB)
    stdPrecision = np.std(precisionNB)
    stdRecall = np.std(recallNB)
    print("Final results NAIVE BAYES----------------------------------")
    print("Average Accuracy:", aveAccuracy)
    print("Average F1:", avef1)
    print("Average Precision:", avePrecision)
    print("Average recall", aveRecall)
    print("STD Accuracy:", stdAccuracy)
    print("STD F1:", stdf1)
    print("STD Precision:", stdPrecision)
    print("STD Recall:", stdRecall)

Final results NAIVE BAYES----------------------------------
Average Accuracy: 0.7847500000000001
Average F1: 0.7862581029215814
Average Precision: 0.7813817488647163
Average recall 0.7915
STD Accuracy: 0.028689065861404392
STD F1: 0.027765564019505858
STD Precision: 0.030979188830704378
STD Recall: 0.02828869031963127


In [78]:
################ pre processing for Scikit models ################    

In [79]:
    trainPos,trainNeg,testPos,testNeg = getPosNegReviews(dataset)

['../TermProject/txt_sentoken/pos\\cv000_29590.txt', '../TermProject/txt_sentoken/pos\\cv001_18431.txt', '../TermProject/txt_sentoken/pos\\cv002_15918.txt', '../TermProject/txt_sentoken/pos\\cv003_11664.txt', '../TermProject/txt_sentoken/pos\\cv004_11636.txt', '../TermProject/txt_sentoken/pos\\cv005_29443.txt', '../TermProject/txt_sentoken/pos\\cv006_15448.txt', '../TermProject/txt_sentoken/pos\\cv007_4968.txt', '../TermProject/txt_sentoken/pos\\cv008_29435.txt', '../TermProject/txt_sentoken/pos\\cv009_29592.txt', '../TermProject/txt_sentoken/pos\\cv010_29198.txt', '../TermProject/txt_sentoken/pos\\cv011_12166.txt', '../TermProject/txt_sentoken/pos\\cv012_29576.txt', '../TermProject/txt_sentoken/pos\\cv013_10159.txt', '../TermProject/txt_sentoken/pos\\cv014_13924.txt', '../TermProject/txt_sentoken/pos\\cv015_29439.txt', '../TermProject/txt_sentoken/pos\\cv016_4659.txt', '../TermProject/txt_sentoken/pos\\cv017_22464.txt', '../TermProject/txt_sentoken/pos\\cv018_20137.txt', '../TermProje

['../TermProject/txt_sentoken/neg\\cv000_29416.txt', '../TermProject/txt_sentoken/neg\\cv001_19502.txt', '../TermProject/txt_sentoken/neg\\cv002_17424.txt', '../TermProject/txt_sentoken/neg\\cv003_12683.txt', '../TermProject/txt_sentoken/neg\\cv004_12641.txt', '../TermProject/txt_sentoken/neg\\cv005_29357.txt', '../TermProject/txt_sentoken/neg\\cv006_17022.txt', '../TermProject/txt_sentoken/neg\\cv007_4992.txt', '../TermProject/txt_sentoken/neg\\cv008_29326.txt', '../TermProject/txt_sentoken/neg\\cv009_29417.txt', '../TermProject/txt_sentoken/neg\\cv010_29063.txt', '../TermProject/txt_sentoken/neg\\cv011_13044.txt', '../TermProject/txt_sentoken/neg\\cv012_29411.txt', '../TermProject/txt_sentoken/neg\\cv013_10494.txt', '../TermProject/txt_sentoken/neg\\cv014_15600.txt', '../TermProject/txt_sentoken/neg\\cv015_29356.txt', '../TermProject/txt_sentoken/neg\\cv016_4348.txt', '../TermProject/txt_sentoken/neg\\cv017_23487.txt', '../TermProject/txt_sentoken/neg\\cv018_21672.txt', '../TermProje

In [80]:
def create_bag_of_words(fileName, reviews):
    word_bag = {}
    stops = stopwords.words('english') + list(string.punctuation)
    for review in reviews:
        with open(review, 'r') as f:
            line = f.read()
            line = line.split(" ")
            if stop:
                line = [word for word in line if word not in set(stops)]
            if alpha:
                line = [word for word in line if word.isalpha()]
            for word in line:
                word_bag[word] = word_bag.get(word, 0) + 1
    with open(fileName, 'w') as f:
        for _ in sorted(word_bag, key=word_bag.get, reverse=True):
            f.write("%s %d\n" % (_, word_bag[_]))
    return word_bag

In [81]:
    nWbag = create_bag_of_words("negBag.txt",trainNeg)
    print(len(nWbag.keys()))
    print(nWbag.keys())
    print(nWbag)

24071


dict_keys(['sweethearts', 'intriguing', 'premise', 'great', 'cast', 'nearly', 'edgy', 'funny', 'problems', 'project', 'traced', 'back', 'writer', 'billy', 'crystal', 'shows', 'lack', 'discipline', 'screenplay', 'typically', 'displays', 'comic', 'relief', 'charity', 'robin', 'williams', 'whoopi', 'goldberg', 'two', 'paragons', 'ignores', 'simple', 'crucial', 'rule', 'screwball', 'comedy', 'work', 'characters', 'must', 'placed', 'rigid', 'social', 'setting', 'context', 'unorthodox', 'antics', 'humorous', 'takes', 'place', 'press', 'junket', 'decorum', 'maintained', 'front', 'reporters', 'promising', 'quickly', 'blows', 'rules', 'thus', 'dissipating', 'tension', 'situation', 'end', 'film', 'lead', 'performers', 'participate', 'huge', 'fight', 'room', 'full', 'journalists', 'looking', 'outbursts', 'mildly', 'amusing', 'structure', 'destroyed', 'cusack', 'catherine', 'play', 'eddie', 'thomas', 'gwen', 'harrison', 'beloved', 'acting', 'duo', 'whose', 'marriage', 'hit', 'skids', 'began', 'see

{'sweethearts': 9, 'intriguing': 33, 'premise': 85, 'great': 314, 'cast': 287, 'nearly': 102, 'edgy': 6, 'funny': 346, 'problems': 129, 'project': 93, 'traced': 5, 'back': 377, 'writer': 60, 'billy': 39, 'crystal': 30, 'shows': 128, 'lack': 100, 'discipline': 1, 'screenplay': 111, 'typically': 6, 'displays': 5, 'comic': 131, 'relief': 34, 'charity': 4, 'robin': 110, 'williams': 75, 'whoopi': 3, 'goldberg': 5, 'two': 679, 'paragons': 1, 'ignores': 5, 'simple': 72, 'crucial': 13, 'rule': 31, 'screwball': 6, 'comedy': 329, 'work': 376, 'characters': 683, 'must': 230, 'placed': 25, 'rigid': 4, 'social': 25, 'setting': 45, 'context': 17, 'unorthodox': 2, 'antics': 18, 'humorous': 32, 'takes': 218, 'place': 228, 'press': 38, 'junket': 6, 'decorum': 2, 'maintained': 7, 'front': 56, 'reporters': 6, 'promising': 29, 'quickly': 107, 'blows': 24, 'rules': 33, 'thus': 32, 'dissipating': 1, 'tension': 48, 'situation': 60, 'end': 408, 'film': 3175, 'lead': 92, 'performers': 24, 'participate': 6, 'hu

In [82]:
    pWbag = create_bag_of_words("posBag.txt",trainPos)
    print(len(pWbag.keys()))
    print(pWbag.keys())
    print(pWbag)

26246


dict_keys(['welcome', 'cinematic', 'trend', 'past', 'couple', 'years', 'make', 'works', 'shakespeare', 'accessible', 'younger', 'audiences', 'vast', 'majority', 'moviegoers', 'contemporizing', 'bard', 'altering', 'settings', 'familiar', 'surroundings', 'prime', 'example', 'things', 'hate', 'basically', 'taming', 'shrew', 'high', 'school', 'successful', 'last', 'adaptation', 'hamlet', 'ethan', 'hawke', 'heir', 'apparent', 'denmark', 'corp', 'setting', 'corporate', 'world', 'new', 'york', 'comes', 'faithful', 'retelling', 'one', 'tragic', 'tragedies', 'stage', 'private', 'prep', 'odin', 'james', 'mekhi', 'phifer', 'general', 'basketball', 'standout', 'player', 'stiles', 'featured', 'well', 'performing', 'ophelia', 'plays', 'desi', 'brable', 'girlfriend', 'whose', 'father', 'also', 'happens', 'dean', 'institution', 'hartnett', 'pearl', 'harbor', 'hand', 'villainy', 'playing', 'scheming', 'hugo', 'sets', 'wheels', 'motion', 'actually', 'filmed', 'ago', 'shelved', 'columbine', 'tragedy', 'm

{'welcome': 30, 'cinematic': 59, 'trend': 10, 'past': 153, 'couple': 157, 'years': 383, 'make': 610, 'works': 184, 'shakespeare': 42, 'accessible': 10, 'younger': 49, 'audiences': 78, 'vast': 17, 'majority': 20, 'moviegoers': 6, 'contemporizing': 1, 'bard': 6, 'altering': 2, 'settings': 11, 'familiar': 72, 'surroundings': 12, 'prime': 25, 'example': 104, 'things': 323, 'hate': 53, 'basically': 52, 'taming': 3, 'shrew': 6, 'high': 214, 'school': 150, 'successful': 94, 'last': 328, 'adaptation': 38, 'hamlet': 19, 'ethan': 23, 'hawke': 8, 'heir': 4, 'apparent': 36, 'denmark': 3, 'corp': 1, 'setting': 68, 'corporate': 12, 'world': 473, 'new': 598, 'york': 113, 'comes': 301, 'faithful': 14, 'retelling': 2, 'one': 2074, 'tragic': 29, 'tragedies': 6, 'stage': 53, 'private': 81, 'prep': 3, 'odin': 4, 'james': 161, 'mekhi': 2, 'phifer': 3, 'general': 83, 'basketball': 11, 'standout': 15, 'player': 28, 'stiles': 7, 'featured': 15, 'well': 696, 'performing': 11, 'ophelia': 2, 'plays': 280, 'desi'

In [83]:
def statistically_split_word_lists(pWbag, nWbag):
    posList = {}
    negList = {}
    thresh = 1.2
    for key in nWbag.keys():
        if key in pWbag.keys():
            posFreq = pWbag[key]
            negFreq = nWbag[key]
            if int(posFreq) >= thresh*int(negFreq):
                posList[key] = pWbag[key]
            elif int(posFreq) < thresh*int(negFreq):
                negList[key] = nWbag[key]
        else:
            negList[key] = nWbag[key]
    for key in pWbag.keys():
        if key not in nWbag.keys():
            posList[key] = pWbag[key]
    print("first filter")
    print(len(posList))
    print(len(negList))
    #determine the boundary size
    max_length=min(len(negList),len(posList))
    print("max_length")
    print(max_length)
    #sort the keys of the map into a list, sorting to get most frequently used words
    sortedNegList = sorted(negList, key=negList.get, reverse=True)[:max_length]
    sortedPosList = sorted(posList, key=posList.get, reverse=True)[:max_length]
    print("sorted max filter")
    print(len(pos_list))
    print(len(neg_list))
    #turn the sorted list into a usable mapping again
    new_neg= {k:negList[k] for k in sortedNegList}
    new_pos = {k:posList[k] for k in sortedPosList}
    print("k filter")
    print(len(new_pos))
    print(len(new_neg))
    return new_neg,new_pos

In [84]:
    neg_keys, pos_keys = statistically_split_word_lists(nWbag, pWbag)

first filter
12792
21411
max_length
12792
sorted max filter
13515
13515
k filter
12792
12792


In [85]:
def generate_svm_featureset(neg_bow,pos_bow):
    review_word_index = []
    for word in neg_bow.keys():
        review_word_index.append(word)
    for word in pos_bow.keys():
        review_word_index.append(word)
    review_word_index = sorted(list(set(review_word_index)))
    return review_word_index

In [86]:
    vocab = generate_svm_featureset(pWbag,nWbag)
    print(vocab)

['aaa', 'aaaaaah', 'aaaahhhs', 'aahs', 'aaliyah', 'aalyah', 'aamir', 'aardman', 'aaron', 'aatish', 'ab', 'aback', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandons', 'abating', 'abba', 'abbe', 'abberation', 'abberline', 'abbots', 'abbott', 'abbreviated', 'abby', 'abc', 'abdomen', 'abducted', 'abductees', 'abduction', 'abductions', 'abe', 'abel', 'aberdeen', 'aberration', 'abetted', 'abetting', 'abeyance', 'abhorrence', 'abhorrent', 'abider', 'abides', 'abigail', 'abiility', 'abilities', 'ability', 'abject', 'ablaze', 'able', 'ably', 'aboard', 'abode', 'abolish', 'abolitionist', 'abolitionists', 'abominable', 'aborginal', 'aboriginal', 'aboriginals', 'abort', 'aborted', 'abortion', 'abortionist', 'abortions', 'abortive', 'aboslutely', 'abound', 'abounded', 'abounds', 'abouts', 'abraded', 'abraham', 'abrahams', 'abrams', 'abrasive', 'abreast', 'abril', 'abroad', 'abrupt', 'abruptly', 'abs', 'absconded', 'absence', 'absences', 'absent', 'absentee', 'absinthe', 'absoloute', 'a

In [87]:
def myTokenize(line, stop=True, negation=True):
    line = line.split(" ")
    stops = stopwords.words('english') + list(string.punctuation)
    if stop:
        line = [word for word in line if word not in set(stops)]
    if alpha:
        line = [word for word in line if word.isalpha()]
#     if negation:
#         line = nltk.sentiment.util.mark_negation(line)
    return line

In [88]:
    cv = CountVectorizer(input='filename', tokenizer=myTokenize, lowercase=True, vocabulary=vocab)
    print(cv)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='filename',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function myTokenize at 0x000002CF53260EA0>,
                vocabulary=['aaa', 'aaaaaah', 'aaaahhhs', 'aahs', 'aaliyah',
                            'aalyah', 'aamir', 'aardman', 'aaron', 'aatish',
                            'ab', 'aback', 'abandon', 'abandoned', 'abandoning',
                            'abandonment', 'abandons', 'abating', 'abba',
                            'abbe', 'abberation', 'abberline', 'abbots',
                            'abbott', 'abbreviated', 'abby', 'abc', 'abdomen',
                            'abducted', 'abductees', ...])


In [89]:
    trainFileNames = trainPos + trainNeg
    testFileNames = testPos + testNeg

In [90]:
    trainSet = cv.fit_transform(trainFileNames)

In [91]:
    testSet = cv.fit_transform(testFileNames)

In [97]:
def compute_accuracies(predictedLabels):
    dev_labels = [1]*800+[0]*800
    yhats = predictedLabels
    accuracy = np.mean(yhats == dev_labels)
    tp = np.sum([yhats[i] == dev_labels[i] and yhats[i] == 1 for i in range(len(yhats))])
    precision = tp / np.sum([yhats[i] == 1 for i in range(len(yhats))])
    recall = tp / (np.sum([yhats[i] != dev_labels[i] and yhats[i] == 0 for i in range(len(yhats))]) + tp)
    f1 = 2 * (precision * recall) / (precision + recall)
    return accuracy, f1, precision, recall

In [98]:
def logisticRegression(trainSet, testSet):
    #consider using strip_accents
    LRclassifier = LogisticRegression()
    labels = [1]*800+[0]*800
    LRclassifier.fit(trainSet,labels)
    predictions = LRclassifier.predict(testSet)
    return predictions

In [99]:
    predictedLabelsLR = logisticRegression(trainSet, testSet)

In [100]:
    testLabels = [1]*800+[0]*800
    compute_accuracies(predictedLabelsLR)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


(0.0, 0.664440734557596, 1.0, 0.4975)